# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,grytviken,-54.2811,-36.5092,2.31,88,23,7.05,GS,1713141712
1,1,port-aux-francais,-49.3500,70.2167,5.47,75,99,13.89,TF,1713141712
2,2,port elizabeth,-33.9180,25.5701,21.17,94,20,5.66,ZA,1713141712
3,3,edinburgh of the seven seas,-37.0676,-12.3116,15.83,71,17,7.12,SH,1713141712
4,4,kerikeri,-35.2268,173.9474,23.84,59,9,2.12,NZ,1713141712


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Configure the map plot
map_plot_4 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1,
    color = "City")
# Display the map plot
map_plot_4


# YOUR CODE HERE

# Display the map
# YOUR CODE HERE

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [18]:
# Narrow down cities that fit criteria and drop any results with null values
#A max temperature lower than 27 degrees but higher than 21
#Wind speed less than 4.5 m/s
#Zero cloudiness
#city_temp = (city_data_df.loc['Max Temp']<27 & city_data_df.loc['Max Temp']>21)
city_temp = city_data_df.loc[(city_data_df['Max Temp'] > 21 ) & ( city_data_df['Max Temp'] < 27) \
& (city_data_df['Wind Speed'] < 4.5 )&(city_data_df['Cloudiness'] == 0)]



# Drop any rows with null values
city_temp = city_temp.dropna()

# Display sample data
city_temp

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
16,16,moab,38.5733,-109.5498,24.78,8,0,2.24,US,1713141713
132,132,hasaki,35.7333,140.8333,22.38,67,0,3.08,JP,1713141721
241,241,fort bragg,35.1390,-79.0060,24.91,35,0,4.12,US,1713141731
297,297,ayden,35.4727,-77.4155,22.16,35,0,4.12,US,1713141727
404,404,lincoln,40.8000,-96.6670,21.53,41,0,4.08,US,1713141641
417,417,griffith,41.5284,-87.4237,21.92,34,0,3.09,US,1713141700
435,435,vasto,42.1243,14.7059,21.12,73,0,2.04,IT,1713141745
446,446,matoury,4.8483,-52.3317,25.98,94,0,1.03,GF,1713141746
469,469,bubaque,11.2833,-15.8333,24.64,83,0,3.58,GW,1713141747
477,477,remire-montjoly,4.9167,-52.2667,26.02,94,0,1.03,GF,1713141748


### Step 3: Create a new DataFrame called `hotel_df`.

In [40]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity

hotel_df = city_temp[['City','Country','Lat','Lng','Humidity']]


# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API

hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df

/var/folders/1v/02kbsv0j4zlb128qzg51jm9m0000gn/T/ipykernel_91273/2830132522.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name'] = ""


,City,Country,Lat,Lng,Humidity,Hotel Name
16,moab,US,38.5733,-109.5498,8,
132,hasaki,JP,35.7333,140.8333,67,
241,fort bragg,US,35.1390,-79.0060,35,
297,ayden,US,35.4727,-77.4155,35,
404,lincoln,US,40.8000,-96.6670,41,
417,griffith,US,41.5284,-87.4237,34,
435,vasto,IT,42.1243,14.7059,73,
446,matoury,GF,4.8483,-52.3317,94,
469,bubaque,GW,11.2833,-15.8333,83,
477,remire-montjoly,GF,4.9167,-52.2667,94,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [37]:
#For each city, use the Geoapify API to find the first hotel located within 10,000 meters of your coordinates.
#Add the hotel name and the country as additional information in the hover message for each city on the map as in the following image:

# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key,
    "limit": 20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat'] 
    lon = row['Lng']
        
        # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle: {lat}, {lon}, {radius}"
    params["bias"] = f"proximity: {lat}, {lon}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url,params=params)
       
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
moab - nearest hotel: No hotel found
hasaki - nearest hotel: No hotel found
fort bragg - nearest hotel: No hotel found
ayden - nearest hotel: No hotel found
lincoln - nearest hotel: No hotel found
griffith - nearest hotel: No hotel found
vasto - nearest hotel: No hotel found
matoury - nearest hotel: No hotel found
bubaque - nearest hotel: No hotel found
remire-montjoly - nearest hotel: No hotel found
galliano - nearest hotel: No hotel found
holland - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
16,moab,US,38.5733,-109.5498,8,No hotel found
132,hasaki,JP,35.7333,140.8333,67,No hotel found
241,fort bragg,US,35.1390,-79.0060,35,No hotel found
297,ayden,US,35.4727,-77.4155,35,No hotel found
404,lincoln,US,40.8000,-96.6670,41,No hotel found
417,griffith,US,41.5284,-87.4237,34,No hotel found
435,vasto,IT,42.1243,14.7059,73,No hotel found
446,matoury,GF,4.8483,-52.3317,94,No hotel found
469,bubaque,GW,11.2833,-15.8333,83,No hotel found
477,remire-montjoly,GF,4.9167,-52.2667,94,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [38]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
hotel_df = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1,
    color = "City")
# Display the map plot
hotel_df
# Display the map
# YOUR CODE HERE

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)